In [34]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv

from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display

import ollama

In [35]:
# Constants 
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [36]:
response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content']

SyntaxError: incomplete input (624666800.py, line 2)

In [37]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [38]:
class Website:
    def __init__(self, url):
        """
        Create this website object from the given url with beutiful soup.
        """
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"

        if soup.body:
        
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get("href") for link in soup.find_all('a')]
        self.links= [link for link in links if link]

    def get_contents(self):
        return "Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
        

In [39]:

ed = Website("https://edwarddonner.com")
# ed = Website("https://oznepalservices.com.au")
print(ed.title)
print(ed.links)

Home - Edward Donner
['https://edwarddonner.com/', 'https://edwarddonner.com/outsmart/', 'https://edwarddonner.com/about-me-and-about-nebula/', 'https://edwarddonner.com/posts/', 'https://edwarddonner.com/', 'https://news.ycombinator.com', 'https://nebula.io/?utm_source=ed&utm_medium=referral', 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html', 'https://patents.google.com/patent/US20210049536A1/', 'https://www.linkedin.com/in/eddonner/', 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/', 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/', 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/', 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/', 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/', 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/', 'https://edwarddonner

In [40]:
link_system_prompt = "You are provided with a list of links found on a webpage.\
You are able to decide which of the links would be most relevant to include in a brochure about the company.\
such as links o an About page, or Careers/Jobs pages."
link_system_prompt += "You should response in JSON as in this example."
link_system_prompt += """
{
    "links": [
    {
    "type": "about page", "url":"https://full.url/goes/here/about"}
    
    },
    {
    "type": "careers page":"https://another.full.url/careers"}
    }
    ]
}
"""

In [41]:
print(link_system_prompt)

You are provided with a list of links found on a webpage.You are able to decide which of the links would be most relevant to include in a brochure about the company.such as links o an About page, or Careers/Jobs pages.You should response in JSON as in this example.
{
    "links": [
    {
    "type": "about page", "url":"https://full.url/goes/here/about"}
    
    },
    {
    "type": "careers page":"https://another.full.url/careers"}
    }
    ]
}



In [42]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [43]:
def messages_for(website):
    return [
        {"role": "system", "content": link_system_prompt},
        {
            "role": "user", "content": get_links_user_prompt(website)
        }
    ]

In [45]:
def summarize(url):
    website = Website(url)
    response = ollama.chat(
    model=MODEL,
    messages=messages_for(website)
    )
    return response['message']['content']

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))
display_summary("https://edwarddonner.com")

Here is the list of relevant links in JSON format:

```
{
  "links": [
    {
      "type": "About page",
      "url": "https://edwarddonner.com/about-me-and-about-nebula/"
    },
    {
      "type": "Blog/Posts",
      "url": "https://edwarddonner.com/posts/"
    },
    {
      "type": "LinkedIn Profile",
      "url": "https://www.linkedin.com/in/eddonner/"
    },
    {
      "type": "Twitter Handle",
      "url": "https://twitter.com/edwarddonner"
    },
    {
      "type": "Facebook Profile",
      "url": "https://www.facebook.com/edward.donner.52"
    }
  ]
}
```

I excluded the following links as they were not relevant to a brochure about the company:

* https://news.ycombinator.com (external news website)
* https://nebula.io/?utm_source=ed&utm_medium=referral (external link to another company's website)
* https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html (external news article)
* https://patents.google.com/patent/US20210049536A1/ (patent application)
* mailto:hello@mygroovydomain.com (email link)
* Terms of Service and other generic links